In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
def cnn_model(features,labels,mode):
    
    input_layer=tf.reshape(features['x'],shape=[-1,28,28,1])
    
    conv1=tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    
    pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
    conv2=tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    
    pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    
    pool2_flat=tf.reshape(pool2,shape=[-1,7*7*64])
    
    dense=tf.layers.dense(inputs=pool2_flat,units=1024,activation=tf.nn.relu)
    
    dropout=tf.layers.dropout(inputs=dense,rate=0.4,training=True)
    
    final_out=tf.layers.dense(inputs=dropout,units=10)
    
    y_pred = tf.nn.softmax(logits=final_out)
    
    y_pred_cls = tf.argmax(y_pred,axis=1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        
        spec = tf.estimator.EstimatorSpec(mode=mode,predictions=y_pred_cls)
        
    else:
        
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=final_out)
        
        loss = tf.reduce_mean(cross_entropy)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        
        train_op = optimizer.minimize(loss=loss,global_step = tf.train.get_global_step())
        
        metrics = { "accuracy":tf.metrics.accuracy(labels,y_pred_cls) }
        
        spec = tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op,eval_metric_ops=metrics)
        
    return spec

    
    

In [5]:
model = tf.estimator.Estimator(model_fn=cnn_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4v5qbsh0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcd78b66748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
mnist=input_data.read_data_sets('mnist/',one_hot=False)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [9]:
x = np.array(mnist.train.images, 'float32')
y= np.array(mnist.train.labels, 'int64')

In [10]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':x},y=y,num_epochs=None,shuffle=True)

In [12]:
model.train(input_fn=train_input_fn,steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4v5qbsh0/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmp4v5qbsh0/model.ckpt.
INFO:tensorflow:loss = 0.047800004, step = 1001
INFO:tensorflow:global_step/sec: 2.35703
INFO:tensorflow:loss = 0.00859764, step = 1101 (42.428 sec)
INFO:tensorflow:global_step/sec: 2.47332
INFO:tensorflow:loss = 0.022395914, step = 1201 (40.432 sec)
INFO:tensorflow:global_step/sec: 2.13626
INFO:tensorflow:loss = 0.005600231, step = 1301 (46.810 sec)
INFO:tensorflow:global_step/sec: 2.52495
INFO:tensorflow:loss = 0.045541592, step = 1401 (39.605 sec)
INFO:tensorflow:global_step/sec: 2.24184
INFO:tensorflow:loss = 0.045795076, step = 1501 (44.608 sec)
INFO:tensorflow:global_step/sec: 1.99259
INFO:tensorf

In [13]:
x1 = np.array(mnist.test.images, 'float32')
y1= np.array(mnist.test.labels, 'int64')

In [14]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':x1},y=y1,num_epochs=1,shuffle=False)

In [15]:
result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-13-12:20:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4v5qbsh0/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-13-12:20:43
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.9928, global_step = 3000, loss = 0.02207031
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: /tmp/tmp4v5qbsh0/model.ckpt-3000


In [16]:
result

{'accuracy': 0.9928, 'loss': 0.02207031, 'global_step': 3000}